In [1]:
import tkinter as tk
from tkinter import ttk
import sqlite3
from tkinter import filedialog
import tkinter.messagebox as messagebox
from PIL import Image, ImageTk
import pytesseract
import io
import os
import shutil
import base64  # Importe o módulo base64

# Variáveis globais para as entradas de dados
local_entry = None
colaborador_entry = None
setor_entry = None
cargo_entry = None
data_solicitacao_entry = None
hora_solicitacao_entry = None
responsavel_entry = None
nro_nf_entry = None
data_compra_entry = None
vl_total_entry = None
itens_comprados_entry = None
vl_item_entry = None

# Função para criar ou conectar ao banco de dados SQLite
def conectar_ao_banco():
    conn = sqlite3.connect("Controle_de_Caixa.db")
    cursor = conn.cursor()

    # Verificar se a tabela já existe, se não, crie-a
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS controle_caixa (
            id INTEGER PRIMARY KEY,
            Local TEXT,
            Colaborador TEXT,
            Setor TEXT,
            Cargo TEXT,
            Data_Solicitacao TEXT,
            Hora_Solicitacao TEXT,
            Responsavel TEXT,
            Nro_NF TEXT,
            Data_Compra TEXT,
            VL_Total TEXT,
            Itens_Comprados TEXT,
            VL_Item TEXT,
            Comprovante BLOB
        )
    ''')

    conn.commit()
    return conn



def abrir_imagem():
    global imagem_path

    file_path = filedialog.askopenfilename(filetypes=[("Imagens", "*.jpg *.jpeg *.png *.bmp *.gif")])
    if file_path:
        imagem = Image.open(file_path)
        imagem.thumbnail((300, 300))  # Redimensiona a imagem para exibição
        imagem = ImageTk.PhotoImage(imagem)
        imagem_label.config(image=imagem)
        imagem_label.image = imagem
        imagem_path = file_path  # Salva o caminho da imagem para uso posterior
        
# Função para selecionar o arquivo de imagem
def selecionar_imagem():
    file_path = filedialog.askopenfilename(filetypes=[("Imagens", "*.jpg *.jpeg *.png *.bmp *.gif")])
    if file_path:
        imagem_entry.delete(0, tk.END)
        imagem_entry.insert(0, file_path)


def mostrar_mensagem(mensagem):
    messagebox.showinfo("Mensagem", mensagem)

# Função para inserir dados no banco de dados e associar a imagem
def inserir_dados():
    # Converte a imagem em base64
    imagem_path = imagem_entry.get()
    if os.path.exists(imagem_path):
        with open(imagem_path, "rb") as f:
            imagem_base64 = base64.b64encode(f.read()).decode()

        dados = (
            local_entry.get(),
            colaborador_entry.get(),
            setor_entry.get(),
            cargo_entry.get(),
            data_solicitacao_entry.get(),
            hora_solicitacao_entry.get(),
            responsavel_entry.get(),
            nro_nf_entry.get(),
            data_compra_entry.get(),
            vl_total_entry.get(),
            itens_comprados_entry.get(),
            vl_item_entry.get(),
            imagem_base64,  # Insira a imagem codificada em base64
        )

        conn.execute('''
            INSERT INTO controle_caixa (
                Local, Colaborador, Setor, Cargo, Data_Solicitacao, Hora_Solicitacao,
                Responsavel, Nro_NF, Data_Compra, VL_Total, Itens_Comprados, VL_Item,
                Comprovante
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', dados)

        conn.commit()
        limpar_campos()

        # Limpar o campo de entrada de imagem
        imagem_entry.delete(0, tk.END)

        # Mostrar uma mensagem de validação
        mostrar_mensagem("Dados inseridos com sucesso!")
    else:
        mostrar_mensagem("O arquivo de imagem não foi encontrado.")
    
# Função para buscar dados no banco de dados pelo ID
def buscar_dados():
    # Obter o ID da imagem selecionada
    selected_item = tree.selection()
    if selected_item:
        image_id = selected_item[0]
        cursor = conn.execute('SELECT Comprovante FROM controle_caixa WHERE id=?', (image_id,))
        data = cursor.fetchone()
        if data:
            # Exibir a imagem no quadro
            imagem = Image.open(io.BytesIO(data[0]))
            imagem.thumbnail((200, 200))
            imagem = ImageTk.PhotoImage(imagem)
            imagem_label.config(image=imagem)
            imagem_label.image = imagem
        else:
            # Se não houver imagem, limpar o quadro
            imagem_label.config(image=None)

# Função para limpar os campos de entrada após a inserção de dados
def limpar_campos():
    for entry in entries:
        entry.delete(0, tk.END)
    imagem_label.config(image=None)

# Função para listar todos os dados do banco de dados
def listar_dados():
    # Limpar qualquer dado existente no Treeview
    for row in tree.get_children():
        tree.delete(row)

    # Executar uma consulta SQL para obter todos os registros
    cursor = conn.execute('SELECT * FROM controle_caixa')
    for row in cursor.fetchall():
        # Inserir os dados no Treeview
        # Certifique-se de que os valores estejam nas colunas corretas
        tree.insert("", "end", values=row[1:])
        
# Função para exibir a imagem selecionada no quadro
def exibir_imagem_selecionada():
    selected_item = tree.selection()
    if selected_item:
        image_id = selected_item[0]
        cursor = conn.execute('SELECT Comprovante FROM controle_caixa WHERE id=?', (image_id,))
        data = cursor.fetchone()
        if data:
            # Exibir a imagem no quadro
            imagem = Image.open(io.BytesIO(data[0]))
            imagem.thumbnail((200, 200))
            imagem = ImageTk.PhotoImage(imagem)
            imagem_label.config(image=imagem)
            imagem_label.image = imagem
        else:
            # Se não houver imagem, limpar o quadro
            imagem_label.config(image=None)

# Configuração da janela principal
root = tk.Tk()
root.title("Controle de Caixa")
root.state('zoomed')  # Abre a janela maximizada


# Conectar ao banco de dados
conn = conectar_ao_banco()

# Cabeçalho da imagem
imagem_label = ttk.Label(root)
imagem_label.place(x=10, y=10, width=400, height=300)

# Entradas de dados
entries = []
labels = ["Local", "Colaborador", "Setor", "Cargo", "Data Solicitação", "Hora Solicitação",
          "Responsável", "Nro NF", "Data Compra", "VL Total", "Itens Comprados", "VL Item"]

row1_x = 10
row2_x = 420
row1_y = 10
row2_y = 10  # Ajustado o alinhamento

# Loop onde você cria as entradas de dados e as associa a variáveis globais
for i, label_text in enumerate(labels[:6]):
    label = ttk.Label(root, text=label_text)
    label.place(x=row1_x, y=row1_y + i * 30, width=100, height=20)
    entry = ttk.Entry(root)
    entry.place(x=row1_x + 110, y=row1_y + i * 30, width=200, height=20)
    entries.append(entry)
    
    # Atribua as entradas às variáveis globais
    if i == 0:
        local_entry = entry
    elif i == 1:
        colaborador_entry = entry
    elif i == 2:
        setor_entry = entry
    elif i == 3:
        cargo_entry = entry
    elif i == 4:
        data_solicitacao_entry = entry
    elif i == 5:
        hora_solicitacao_entry = entry

for i, label_text in enumerate(labels[6:]):
    label = ttk.Label(root, text=label_text)
    label.place(x=row2_x, y=row2_y + i * 30, width=100, height=20)
    entry = ttk.Entry(root)
    entry.place(x=row2_x + 110, y=row2_y + i * 30, width=200, height=20)
    entries.append(entry)
    
    # Atribua as entradas às variáveis globais
    if i == 0:
        responsavel_entry = entry
    elif i == 1:
        nro_nf_entry = entry
    elif i == 2:
        data_compra_entry = entry
    elif i == 3:
        vl_total_entry = entry
    elif i == 4:
        itens_comprados_entry = entry
    elif i == 5:
        vl_item_entry = entry

# Botões à direita dos campos de seleção
button_x = 1000

inserir_imagem_button = ttk.Button(root, text="Selecionar Imagem", command=selecionar_imagem)
inserir_imagem_button.place(x=button_x, y=10, width=200, height=30)  # Ajustado o alinhamento

inserir_dados_button = ttk.Button(root, text="Inserir Dados", command=inserir_dados)
inserir_dados_button.place(x=button_x, y=50, width=200, height=30)

listar_dados_button = ttk.Button(root, text="Listar Dados", command=listar_dados)
listar_dados_button.place(x=button_x, y=90, width=200, height=30)

buscar_dados_button = ttk.Button(root, text="Buscar Dados", command=buscar_dados)
buscar_dados_button.place(x=button_x, y=130, width=200, height=30)

# Botão para exibir a imagem selecionada no quadro
exibir_imagem_button = ttk.Button(root, text="Exibir Imagem Selecionada", command=exibir_imagem_selecionada)
exibir_imagem_button.place(x=button_x, y=170, width=200, height=30)

# Adicionar um quadro para exibir a imagem com borda
imagem_frame = ttk.Frame(root, borderwidth=1, relief="solid")
imagem_frame.place(x=750, y=10, width=200, height=200)

# Rótulo (label) para "Local Imagem"
local_imagem_label = ttk.Label(root, text="Local Imagem")
local_imagem_label.place(x=row1_x, y=row1_y + 6 * 30, width=100, height=20)

# Entrada de dados para o endereço da imagem
imagem_entry = ttk.Entry(root)
imagem_entry.place(x=row1_x + 110, y=row1_y + 6 * 30, width=200, height=20)

# Treeview para listar dados
tree = ttk.Treeview(root, columns=labels[1:], show="headings")
for i, label_text in enumerate(labels[1:]):
    tree.heading(label_text, text=label_text)
    tree.column(label_text, width=100)
tree.place(x=10, y=220, width=1300, height=200)

# Associe a função exibir_imagem_selecionada a um evento de seleção no Treeview
tree.bind("<<TreeviewSelect>>", exibir_imagem_selecionada)

# Adicionar um quadro para exibir a imagem com borda
imagem_frame = ttk.Frame(root, borderwidth=1, relief="solid")
imagem_frame.place(x=750, y=10, width=200, height=200)




root.mainloop()
